In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from ConvBN1d import ConvBN
from LinearBN import LinearBN
from source import train, test

In [2]:
def load_ucr(file):
    data = np.loadtxt(file)
    X = data[:, 1:]
    y = data[:, 0]
    y = np.where(y == 1, 1, 0)  # convert labels to 0/1
    return X, y

# Adjust file paths to your local files
X_train, y_train = load_ucr("FordA_TRAIN.txt")
X_test, y_test = load_ucr("FordA_TEST.txt")

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (3601, 500), Test shape: (1320, 500)


In [3]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [4]:
mean = X_train_tensor.mean()
std = X_train_tensor.std()
X_train_tensor = (X_train_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
class LReLU(nn.Module):
    def __init__(self):
        super(LReLU, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(5.0)) 
    def forward(self, x):
        return torch.nn.functional.relu(self.alpha*x)

In [6]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1_out = 16
        self.conv1_size = 15
        self.conv1_padding = 7


        self.conv2_out = 16
        self.conv2_size = 15
        self.conv2_padding = 7

        self.conv3_out = 25
        self.conv3_size = 13
        self.conv3_padding = 6

        self.fc1_out = 2

        self.q = 1e-6
        self.bias_trick_par = nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block

        self.block1 = ConvBN(in_channels=1, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .05, bias_par_init=0.0015)
        self.block2 = ConvBN(in_channels=self.conv1_out, out_channels=self.conv2_out, kernel_size=self.conv2_size, padding=self.conv2_padding, std = .15, bias_par_init=0.0015)
        self.block3 = ConvBN(in_channels=self.conv2_out, out_channels=self.conv3_out, kernel_size=self.conv3_size, padding=self.conv3_padding, std = .15, bias_par_init=0.0015)
               
        
        # torch.manual_seed(0)
        self.w2 = nn.Parameter(torch.randn(self.conv3_out * 500, self.fc1_out))
        nn.init.normal_(self.w2, mean=0.0, std=.6)

        self.dropout = nn.Dropout(0.5)

        self.relu = LReLU()




    def forward(self, x):
        
        x = self.relu(self.block1(x))
        x = self.relu(self.block2(x))
        x = self.relu(self.block3(x))
        
        x = x.view(x.size(0), -1)
        
        # x = self.relu(self.block3(x))
        # x = self.dropout(x)

        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

    def custom_round(self, n):
        remainder = n % 1000
        base = n - remainder
        if remainder >= 101:
            return base + 1000
        elif remainder <= 100:
            return base

    def init_hdc(self, ratio, seed):
        if not isinstance(ratio, (tuple, int)):
            raise TypeError("ratio must be a tuple of size 4 or and integer")

        elif isinstance(ratio, (int)):
            ratio = (ratio, ratio, ratio, ratio)
            
        if not isinstance(seed, (tuple)):
            raise TypeError("seed must be a tuple of size 4")
        
        self.block1.init_hdc(ratio = ratio[0], seed = seed[0])
        self.block2.init_hdc(ratio = ratio[1], seed = seed[1])
        self.block3.init_hdc(ratio = ratio[2], seed = seed[2])
                
        self.n_last = self.w2.size(0)
        self.nHDC_last = int(self.custom_round(ratio[3] * self.n_last)) if ratio[3]<1000 else int(ratio[3])
        torch.manual_seed(seed[3])
        self.g = (torch.randn(self.w2.size(0), self.nHDC_last, device=self.w2.device)).to(torch.half)
        self.wg = torch.sign(torch.matmul(self.g.t(), self.w2.to(torch.half)))


    def hdc(self, x):
        x = self.relu(self.block1.hdc(x))
        x = self.relu(self.block2.hdc(x))
        x = self.relu(self.block3.hdc(x))

        x = x.view(x.size(0), -1)
        
        x = x + self.bias_trick_par
        x = torch.sign(torch.matmul(x.to(torch.half), self.g))

        return x
        
    def classification_layer(self, x):
        x = x @ self.wg
        return x

In [7]:
from tqdm import tqdm
import time
from torch.utils.data import Subset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


torch.cuda.empty_cache()
model = Network().to(device)
model.load_state_dict(torch.load('FordA_GNet_Training_92.35.pth', weights_only = True))

<All keys matched successfully>

In [8]:
model.to(torch.half).to(device)
model.eval()
n_splits = 10
split_size = len(test_dataset) // n_splits  # 10000 // 20 = 500
print(split_size)

times = []
num_workers=2
pin_memory=True
hyperdims = range(5000, 21000, 1000)
print(len(hyperdims))
NHDC = np.zeros((len(hyperdims), 4))
accuracies = np.zeros((len(hyperdims), n_splits))
for i, nhdc in enumerate(hyperdims):
    indices = list(range(len(test_dataset)))
    np.random.seed(42)
    np.random.shuffle(indices)  # or random.shuffle(indices)
    for split_idx in tqdm(range(n_splits)):
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_indices = indices[start_idx:end_idx]
        split_subset = Subset(test_dataset, split_indices)
        split_loader = torch.utils.data.DataLoader(split_subset, batch_size=15, shuffle=False,
                                                   num_workers=num_workers, pin_memory=pin_memory)
        torch.manual_seed(split_idx)
        random_seeds = tuple(torch.randint(0, 1000, (1,)).item() for _ in range(4))
        torch.cuda.empty_cache()
        
        model.init_hdc(nhdc, random_seeds)
        NHDC[i] = [model.block1.nHDC, model.block2.nHDC, model.block3.nHDC, model.nHDC_last]
        correct = 0
        total = 0
    
        t0 = time.time()
        with torch.no_grad():
            for images, labels in (split_loader):
                images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                output = model.hdc(images.to(torch.half))
                output = model.classification_layer(output.to(torch.half))
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        t1 = time.time()
    
        acc = 100 * correct / total
        dt = t1 - t0
    
        accuracies[i, split_idx] = acc
        times.append(dt)
    
    print(f'Block1: {model.block1.nHDC}, Block2: {model.block2.nHDC}, Block3: {model.block3.nHDC}, Classification Layer: {model.nHDC_last}, Avg Acc: {np.mean(accuracies[i]):.2f}%')


132
16


100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


Block1: 5000, Block2: 5000, Block3: 5000, Classification Layer: 5000, Avg Acc: 81.52%


100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


Block1: 6000, Block2: 6000, Block3: 6000, Classification Layer: 6000, Avg Acc: 81.67%


...

Block1: 18000, Block2: 18000, Block3: 18000, Classification Layer: 18000, Avg Acc: 88.64%


100%|██████████| 10/10 [00:33<00:00,  3.33s/it]


Block1: 19000, Block2: 19000, Block3: 19000, Classification Layer: 19000, Avg Acc: 88.86%


100%|██████████| 10/10 [00:35<00:00,  3.54s/it]

Block1: 20000, Block2: 20000, Block3: 20000, Classification Layer: 20000, Avg Acc: 89.24%


In [9]:
np.mean(accuracies, axis=1), np.std(accuracies, axis=1)

(array([81.51515152, 81.66666667, 82.04545455, 84.16666667, 83.93939394,
        85.22727273, 85.45454545, 85.90909091, 86.21212121, 87.27272727,
        87.27272727, 88.10606061, 89.62121212, 88.63636364, 88.86363636,
        89.24242424]),
 array([4.39393939, 3.26373625, 3.92260408, 4.35787182, 2.11036186,
        4.30220329, 5.58554806, 2.85878216, 3.35050673, 2.87080232,
        3.06420431, 1.8572198 , 2.97390703, 2.34726263, 1.62658413,
        2.76904044]))

In [10]:
from scipy.io import savemat
savemat('FordA_EHDGNet.mat',{'FordA_EHDGNet':accuracies})

In [11]:
savemat('FordA_nHD.mat', {'FordA_nHD':NHDC})